<a href="https://colab.research.google.com/github/PatriciaOliveiraMoitinho/pas-platform/blob/main/Rob%C3%B4_de_Estudos_Universal_v11_0_(com_Download_em_HTML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# =======================================================================================
#      ROBÔ DE ESTUDOS v17.0 - COM HIERARQUIA CORRETA DE LISTAS
# =======================================================================================

# =======================================================================================
#      >>> SEU PAINEL DE CONTROLE <<<
#
# Altere SOMENTE as informações aqui para cada novo estudo.
# Depois, aperte o botão "Play" (▶️) para rodar a célula.
# O campo para colar o texto aparecerá na SAÍDA da célula.
# =======================================================================================

# 1. Coloque aqui o nome do seu novo assunto.
nome_do_assunto = "Análise da Lei nº 8.112/1990"

# 2. Coloque aqui a lista com os caminhos de TODOS os seus arquivos de fontes.
#    (Clique 2x para editar)
caminhos_fontes = [
    # Exemplo: "/content/drive/MyDrive/TCU/Direito Administrativo/Fontes/Fonte_1.pdf",
    # Adicione seus caminhos aqui. Se não tiver, deixe a lista vazia: []
]

# --- CONTROLE DAS CHAVES DE API ---
# 3. Coloque aqui o NOME EXATO do seu 'secret' da chave do Google AI (Gemini).
#    (O nome que você salvou no menu 🔑 do Colab)
nome_secret_google_ai = "minha_chave_api"

# 4. Coloque aqui o NOME EXATO do seu 'secret' da chave da API de Busca do Google.
#    (O nome que você salvou como 'GOOGLE_API_KEY')
nome_secret_google_search_key = "GOOGLE_API_KEY"

# 5. Coloque aqui o NOME EXATO do seu 'secret' do ID do Mecanismo de Busca.
#    (O nome que você salvou como 'GOOGLE_CX')
nome_secret_google_search_cx = "GOOGLE_CX"


# =======================================================================================
#      >>> FIM DO PAINEL DE CONTROLE <<<
#
# Daqui para baixo, o robô trabalha sozinho.
# =======================================================================================


# PARTE 1: INSTALAÇÃO E CONFIGURAÇÃO DO AMBIENTE
# ---------------------------------------------------------------------------------------
print("--- PARTE 1: CONFIGURANDO O AMBIENTE ---")
import os
import re
import html
import markdown
import sys
from datetime import datetime
from google.colab import drive, files
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# Tenta importar as bibliotecas e, se não conseguir, instala.
try:
    import google.generativeai as genai
    from googleapiclient.discovery import build
    import fitz  # PyMuPDF
except ImportError:
    print("-> Instalando as bibliotecas necessárias...")
    !pip install -q -U google-generativeai google-api-python-client PyMuPDF ipywidgets
    import google.generativeai as genai
    from googleapiclient.discovery import build
    import fitz
    print("-> Bibliotecas instaladas com sucesso!")

chaves_configuradas = False
try:
    from google.colab import userdata
    # Carrega as chaves do Colab Secrets
    chave_google_ai = userdata.get(nome_secret_google_ai)
    chave_google_search = userdata.get(nome_secret_google_search_key)
    id_mecanismo_busca = userdata.get(nome_secret_google_search_cx)

    if not all([chave_google_ai, chave_google_search, id_mecanismo_busca]):
        raise ValueError("Uma ou mais chaves não foram encontradas ou estão vazias. Verifique os nomes no Painel de Controle.")

    genai.configure(api_key=chave_google_ai)
    print("✅ Chave da API do Google AI (Gemini) configurada.")
    print("✅ Chave da API de Busca (Google Custom Search) configurada.")
    print("✅ ID do Mecanismo de Busca configurado.")
    chaves_configuradas = True

except Exception as e:
    print(f"\n❌ ERRO CRÍTICO: {e}")
    print("\n--- AÇÃO NECESSÁRIA ---")
    print("O robô não encontrou suas 'senhas' (chaves de API). Verifique duas coisas:")
    print("1. Se os NOMES das chaves no 'Painel de Controle' estão EXATAMENTE iguais aos nomes que você salvou no menu de chaves (🔑 Secrets) do Colab.")
    print("2. Se os botões ao lado dos nomes das chaves no menu 'Secrets' estão LIGADOS.")
    print("\nDepois de verificar, execute esta célula de código novamente.")


if chaves_configuradas:
    try:
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive conectado.")
    except Exception as e:
        print(f"❌ ERRO ao conectar ao Drive: {e}")
        sys.exit("O robô foi interrompido porque não conseguiu se conectar ao Google Drive.")

    print("--- Configuração do ambiente concluída! ---\n")


    # PARTE 2: LEITURA DOS ARQUIVOS DE FONTES
    # ---------------------------------------------------------------------------------------
    print("--- PARTE 2: LENDO OS ARQUIVOS DO SEU DRIVE ---")
    def ler_pdf(caminho_do_arquivo):
        try:
            nome_arquivo = os.path.basename(caminho_do_arquivo)
            print(f"Lendo o arquivo: {nome_arquivo}...")
            with fitz.open(caminho_do_arquivo) as doc:
                texto_completo = ""
                for pagina in doc:
                    texto_completo += pagina.get_text()
            print(f"-> Leitura de '{nome_arquivo}' concluída!")
            return texto_completo
        except Exception as e:
            print(f"!!!!!! ERRO AO LER O ARQUIVO: {os.path.basename(caminho_do_arquivo)} !!!!!!")
            print(f"Detalhe do erro: {e}")
            return ""

    if caminhos_fontes:
        texto_fontes_combinado = ""
        for caminho in caminhos_fontes:
            nome_curto = os.path.basename(caminho)
            texto_fontes_combinado += f"\n\n--- INÍCIO DA FONTE: {nome_curto} ---\n\n"
            texto_fontes_combinado += ler_pdf(caminho)
            texto_fontes_combinado += f"\n\n--- FIM DA FONTE: {nome_curto} ---\n\n"

        print(f"\n✅ Todos os {len(caminhos_fontes)} arquivos de fontes foram lidos e estão na memória!")
    else:
        texto_fontes_combinado = "Nenhuma fonte de arquivo foi fornecida."
        print("\n🟡 Nenhum arquivo de fonte foi listado. A análise usará apenas o conhecimento geral da IA.")

    print("--- Leitura dos arquivos concluída! ---\n")


    # PARTE 3: DEFINIÇÃO DE TODAS AS FERRAMENTAS DO ROBÔ
    # ---------------------------------------------------------------------------------------
    print("--- PARTE 3: PREPARANDO TODAS AS FERRAMENTAS DO ROBÔ ---\n")
    try:
        model = genai.GenerativeModel('models/gemini-pro-latest')
        print("✅ Ferramenta de IA ('models/gemini-pro-latest') configurada!")
    except Exception as e:
        model = None
        print(f"❌ Falha ao configurar a ferramenta de IA: {e}")

    def etapa_1_mapear_fontes(trecho_da_lei, lista_de_caminhos):
        print("\n-> Acionando Ferramenta 1: Mapeamento de Fontes (um arquivo por vez)...")
        if not lista_de_caminhos:
             print("  -> Nenhuma fonte para mapear. Pulando esta etapa.")
             return "Nenhuma fonte local foi usada para a análise."
        resultados_combinados = ""
        for caminho_arquivo in lista_de_caminhos:
            nome_arquivo = os.path.basename(caminho_arquivo)
            print(f"\n  -> Analisando a fonte: {nome_arquivo}")
            conteudo_do_arquivo = ler_pdf(caminho_arquivo)
            if not conteudo_do_arquivo.strip():
                print(f"  -> Fonte '{nome_arquivo}' está vazia ou não pôde ser lida. Pulando.")
                continue
            prompt = f"""Você é um assistente de pesquisa. Sua tarefa é analisar o 'TRECHO DA LEI' abaixo.
Depois, leia o 'CONTEÚDO DA FONTE' e encontre APENAS os parágrafos que são diretamente relevantes para os artigos e tópicos mencionados na lei.
Extraia esses parágrafos relevantes. Se nenhum parágrafo for relevante, apenas diga "Nenhum conteúdo relevante encontrado nesta fonte."

TRECHO DA LEI:
{trecho_da_lei}

CONTEÚDO DA FONTE ({nome_arquivo}):
{conteudo_do_arquivo}"""
            try:
                response = model.generate_content(prompt, request_options={"timeout": 600})
                resultados_combinados += f"\n\n--- ANÁLISE DA FONTE: {nome_arquivo} ---\n\n"
                resultados_combinados += response.text
                print(f"  -> Análise de '{nome_arquivo}' concluída.")
            except Exception as e:
                print(f"  -> ❌ Erro ao analisar a fonte '{nome_arquivo}': {e}")
                resultados_combinados += f"\n\n--- ERRO NA FONTE: {nome_arquivo} ---\n\nOcorreu um erro ao processar esta fonte.\n"
        print("\n-> Ferramenta 1 finalizou o mapeamento de todas as fontes.")
        return resultados_combinados

    def etapa_2_analise_juridica(trecho_da_lei, extrato_das_fontes):
        print("\n-> Acionando Ferramenta 2: Análise Jurídica Central...")
        prompt = f"Você é um jurista e professor. Faça uma análise técnica e detalhada do 'TEXTO DA LEI' abaixo, artigo por artigo, usando o 'EXTRATO DAS FONTES' como apoio.\n\nTEXTO DA LEI:\n{trecho_da_lei}\n\nEXTRATO DAS FONTES:\n{extrato_das_fontes}"
        response = model.generate_content(prompt, request_options={"timeout": 1000}); print("-> Ferramenta 2 finalizou."); return response.text

    def etapa_final_aula_completa(mapa_das_fontes, analise_tecnica):
        print("\n-> Acionando Ferramenta 3: A Aula Completa...")
        prompt = f"**PERSONA:** Você é um **Professor da ENAP** especialista em **Linguagem Simples**. **MISSÃO:** Una o 'MAPA DAS FONTES' e a 'ANÁLISE TÉCNICA' em uma **AULA COMPLETA, LONGA E DETALHADA**. **REGRAS DE FORMATAÇÃO:** Use marcadores (●, ○, ▪) para listas. Para negrito, use **texto**. NUNCA use `* **` juntos. **ESTRUTURA:** 1. Introdução; 2. Análise Artigo por Artigo; 3. O que os Especialistas Dizem; 4. Resumo Final.\n\nMAPA DAS FONTES:\n{mapa_das_fontes}\n\nANÁLISE TÉCNICA:\n{analise_tecnica}"
        response = model.generate_content(prompt, request_options={"timeout": 1000}); print("-> Ferramenta 3 concluiu a aula."); return response.text

    def etapa_feynman(aula_completa):
        print("\n-> Acionando Ferramenta 4: Técnica de Feynman...")
        prompt = f"**PERSONA:** Você é um Professor da ENAP. **MISSÃO:** Leia a 'AULA COMPLETA' e aplique a Técnica de Feynman. **REGRAS DE FORMATAÇÃO:** Use marcadores (●) para listas. Para negrito, use **texto**. NUNCA use `* **` juntos. **ESTRUTURA:** 1. Identifique os 2-3 conceitos mais complexos; 2. Para cada um, explique de forma ultra simples; 3. Crie uma analogia do dia a dia; 4. Aponte o ponto de atenção para concursos.\n\nAULA COMPLETA:\n{aula_completa}"
        response = model.generate_content(prompt, request_options={"timeout": 1000}); print("-> Ferramenta 4 concluiu a simplificação."); return response.text

    def etapa_glossario(aula_completa):
        print("\n-> Acionando Ferramenta 5: Glossário Remissivo...")
        prompt = f"**PERSONA:** Você é um Professor da ENAP lexicógrafo. **MISSÃO:** Leia a 'AULA COMPLETA', identifique termos técnicos e crie um glossário em ordem alfabética com definições simples e diretas. **REGRAS DE FORMATAÇÃO:** Use **Termo:** para o item e depois a definição. NUNCA use `* **` juntos.\n\nAULA COMPLETA:\n{aula_completa}"
        response = model.generate_content(prompt, request_options={"timeout": 1000}); print("-> Ferramenta 5 concluiu o glossário."); return response.text

    def etapa_gerador_questoes(aula_completa):
        print("\n-> Acionando Ferramenta 6: Gerador de Questões...")
        prompt = f"**PERSONA:** Você é um examinador da banca Cebraspe. **MISSÃO:** Leia a 'AULA COMPLETA' e crie 5 questões inéditas (Certo ou Errado) baseadas em 'pegadinhas' do texto. **REGRAS DE FORMATAÇÃO:** Numere as questões. Para o gabarito, use **Questão X:** seguido de Certo/Errado e a justificativa. NUNCA use `* **` juntos.\n\nAULA COMPLETA:\n{aula_completa}"
        response = model.generate_content(prompt, request_options={"timeout": 1000}); print("-> Ferramenta 6 concluiu as questões."); return response.text

    def etapa_mapa_mental(aula_completa):
        print("\n-> Acionando Ferramenta 7: Mapa Mental...")
        prompt = f"**PERSONA:** Você é um Professor da ENAP especialista em aprendizagem visual. **MISSÃO:** Transforme a 'AULA COMPLETA' em um mapa mental em texto. **REGRAS DE FORMATAÇÃO:** Use marcadores (●, ○, ▪) e recuo para criar a hierarquia. Use apenas palavras-chave. NUNCA use `* **` juntos.\n\nAULA COMPLETA:\n{aula_completa}"
        response = model.generate_content(prompt, request_options={"timeout": 1000}); print("-> Ferramenta 7 concluiu o mapa."); return response.text

    def etapa_pesquisa_jurisprudencia(texto_analisado):
        print("\n-> Acionando Ferramenta 8: Pesquisa de Jurisprudência...")
        try:
            prompt_termos = f"Com base no seguinte texto, gere 3 termos de busca curtos e eficazes para encontrar jurisprudência recente no Brasil. Responda apenas com os termos, separados por vírgula.\n\nTEXTO: '{texto_analisado[:1000]}...'"
            response_termos = model.generate_content(prompt_termos)
            termos_de_busca = response_termos.text.strip().split(',')
            print(f"-> Termos de busca gerados pela IA: {termos_de_busca}")
            service = build("customsearch", "v1", developerKey=chave_google_search)
            resultados_finais = ""
            for termo in termos_de_busca:
                termo = termo.strip()
                if not termo: continue
                print(f"-> Pesquisando por: '{termo}'...")
                resultados_finais += f"\n\n**Resultados da pesquisa para: '{termo}'**\n"
                res = service.cse().list(q=termo, cx=id_mecanismo_busca, num=3).execute()
                if 'items' in res and res['items']:
                    for result in res['items']:
                        titulo = result.get('title', 'N/A')
                        link = result.get('link', '#')
                        snippet = result.get('snippet', 'N/A').replace('\n', '')
                        resultados_finais += f"● **{titulo}**: {snippet} ([Link]({link}))\n"
                else:
                    resultados_finais += f"● Nenhuma jurisprudência relevante encontrada para este termo.\n"
            print("-> Ferramenta 8 (Pesquisa) concluiu o trabalho.")
            return resultados_finais
        except Exception as e:
            print(f"❌ Erro na ferramenta 'Pesquisa de Jurisprudência': {e}")
            return f"Ocorreu um erro durante a pesquisa online: {e}"

    def etapa_tabela_comparativa(aula_completa, lista_de_fontes):
        print("\n-> Acionando Ferramenta 9: Tabela Comparativa...")
        nomes_autores = [os.path.basename(caminho).replace('.pdf', '') for caminho in lista_de_fontes] if lista_de_fontes else ["Doutrina Geral"]
        prompt = f"**PERSONA:** Você é um **Professor da ENAP**. **MISSÃO:** Leia a 'AULA COMPLETA' e crie uma **tabela comparativa em formato HTML**. **REGRAS:** O resultado deve ser **apenas o código HTML da tabela (`<table>...</table>`)**. A tabela deve comparar 3 a 5 conceitos-chave, com os nomes dos autores nas colunas.\n\nAULA COMPLETA:\n{aula_completa}\n\nLISTA DE AUTORES:\n{', '.join(nomes_autores)}"
        response = model.generate_content(prompt, request_options={"timeout": 1000})
        print("-> Ferramenta 9 (Tabela) concluiu o trabalho.")
        return response.text

    # --- FUNÇÃO DE FORMATAÇÃO CORRIGIDA ---
    def formatar_listas_aninhadas(texto_markdown):
        """
        Esta função corrige a formatação de listas no markdown gerado pela IA.
        Ela converte os marcadores personalizados (●, ○, ▪) em listas aninhadas
        de Markdown padrão (*), que são corretamente convertidas para tags <ul> e <li>
        em HTML.
        """
        # A ordem de execução é importante: do mais aninhado (interno) para o menos aninhado.

        # Substitui os marcadores de 3º nível (▪) por um marcador padrão com 8 espaços de recuo.
        # O \n no início da substituição garante que cada item da lista comece em uma nova linha.
        # O \s* antes e depois do marcador limpa espaços em branco extras.
        texto_formatado = re.sub(r'\s*▪\s*', r'\n        * ', texto_markdown)

        # Faz o mesmo para os marcadores de 2º nível (○), com 4 espaços de recuo.
        texto_formatado = re.sub(r'\s*○\s*', r'\n    * ', texto_formatado)

        # E para os marcadores de 1º nível (●), sem recuo.
        texto_formatado = re.sub(r'\s*●\s*', r'\n* ', texto_formatado)

        return texto_formatado

    print("✅ Todas as ferramentas do robô estão prontas!\n")


    # PARTE 4: ÁREA DE INTERAÇÃO - COLE SEU TEXTO AQUI
    # ---------------------------------------------------------------------------------------
    textarea = widgets.Textarea(
        placeholder='Cole aqui o texto que você quer analisar...',
        description='Texto:',
        layout={'width': '99%', 'height': '250px'}
    )
    button = widgets.Button(
        description="Analisar Texto",
        button_style='success',
        tooltip='Clique aqui para iniciar a análise do texto acima',
        icon='cogs'
    )
    output_area = widgets.Output()

    def iniciar_analise(b):
        with output_area:
            clear_output()
            button.disabled = True
            print("--- INICIANDO ANÁLISE ---")

        texto_para_analisar = textarea.value
        if not texto_para_analisar.strip():
            with output_area:
                print("\n❌ ATENÇÃO: A caixa de texto está vazia. Por favor, cole um texto para analisar.")
            button.disabled = False
            return

        with output_area:
            try:
                print("\n✅ Texto recebido! Iniciando a análise completa...")
                resultado_etapa_1 = etapa_1_mapear_fontes(texto_para_analisar, caminhos_fontes)
                resultado_etapa_2 = etapa_2_analise_juridica(texto_para_analisar, resultado_etapa_1)
                resultado_aula = etapa_final_aula_completa(resultado_etapa_1, resultado_etapa_2)
                resultado_jurisprudencia = etapa_pesquisa_jurisprudencia(texto_para_analisar)
                resultado_tabela_html = etapa_tabela_comparativa(resultado_aula, caminhos_fontes)
                resultado_tabela_html = re.sub(r'^```html\n', '', resultado_tabela_html.strip())
                resultado_tabela_html = re.sub(r'\n```$', '', resultado_tabela_html)
                resultado_feynman = etapa_feynman(resultado_aula)
                resultado_glossario = etapa_glossario(resultado_aula)
                resultado_questoes = etapa_gerador_questoes(resultado_aula)
                resultado_mapa = etapa_mapa_mental(resultado_aula)

                # --- MUDANÇA AQUI: Aplicando a nova formatação em todos os textos ---
                print("\n-> Aplicando formatação avançada de listas...")
                resultado_aula = formatar_listas_aninhadas(resultado_aula)
                resultado_jurisprudencia = formatar_listas_aninhadas(resultado_jurisprudencia)
                resultado_feynman = formatar_listas_aninhadas(resultado_feynman)
                resultado_glossario = formatar_listas_aninhadas(resultado_glossario)
                resultado_questoes = formatar_listas_aninhadas(resultado_questoes)
                resultado_mapa = formatar_listas_aninhadas(resultado_mapa) # CORREÇÃO APLICADA AQUI

                # PARTE 5: GERAR E BAIXAR O ARQUIVO FINAL EM HTML
                print("\n\n--- PREPARANDO SEU ARQUIVO HTML PARA DOWNLOAD ---")

                texto_analisado_seguro = html.escape(texto_para_analisar)
                texto_analisado_formatado = f"<pre>{texto_analisado_seguro}</pre>"

                html_aula = markdown.markdown(resultado_aula, extensions=['fenced_code', 'tables'])
                html_jurisprudencia = markdown.markdown(resultado_jurisprudencia, extensions=['fenced_code', 'tables'])
                html_feynman = markdown.markdown(resultado_feynman, extensions=['fenced_code', 'tables'])
                html_glossario = markdown.markdown(resultado_glossario, extensions=['fenced_code', 'tables'])
                html_questoes = markdown.markdown(resultado_questoes, extensions=['fenced_code', 'tables'])

                # --- CORREÇÃO AQUI: Simplificando a criação do HTML para o Mapa Mental ---
                html_mapa = markdown.markdown(resultado_mapa, extensions=['fenced_code', 'tables'])

                timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
                nome_seguro_para_arquivo = re.sub(r'[\\/*?:"<>|]', "", nome_do_assunto)
                nome_do_arquivo = f"{nome_seguro_para_arquivo}_{timestamp}.html"

                conteudo_html = f"""
<!DOCTYPE html><html lang="pt-BR"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><title>{nome_do_assunto}</title>
<style>
    body {{ font-family: sans-serif; line-height: 1.6; margin: 0; padding: 20px; background-color: #f4f4f4; color: #333; }}
    .container {{ max-width: 900px; margin: auto; background: #fff; padding: 30px; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }}
    h1, h2 {{ color: #0056b3; border-bottom: 2px solid #0056b3; padding-bottom: 10px; }} h1 {{ text-align: center; font-size: 2.5em; }} h2 {{ font-size: 1.8em; margin-top: 40px; }}
    .card {{ background: #f9f9f9; border-left: 5px solid #0056b3; margin: 20px 0; padding: 20px; border-radius: 5px; }}
    p {{ text-align: justify; }}
    ul, ol {{ padding-left: 40px; text-align: justify; }}
    li {{ margin-bottom: 10px; }}
    table {{ width: 100%; border-collapse: collapse; margin-top: 20px; }} th, td {{ padding: 12px; border: 1px solid #ddd; text-align: left; }} th {{ background-color: #0056b3; color: white; }}
    tr:nth-child(even) {{ background-color: #f2f2f2; }} blockquote {{ background: #e9ecef; border-left: 5px solid #6c757d; margin: 20px 0; padding: 15px; font-style: italic; }}
    strong, b {{ color: #0056b3 !important; }} pre {{ white-space: pre-wrap; word-wrap: break-word; font-family: inherit; font-size: inherit; margin: 0; padding: 0; background-color: transparent; border: none; }}
</style></head><body><div class="container">
<h1>Análise Completa: {nome_do_assunto.replace('_', ' ')}</h1>
<div class="card"><h2>PARTE 1: TEXTO-BASE ANALISADO</h2><blockquote>{texto_analisado_formatado}</blockquote></div>
<div class="card"><h2>PARTE 2: AULA COMPLETA</h2>{html_aula}</div><div class="card"><h2>PARTE 3: TABELA COMPARATIVA DE DOUTRINADORES</h2>{resultado_tabela_html}</div>
<div class="card"><h2>PARTE 4: JURISPRUDÊNCIA RECENTE</h2>{html_jurisprudencia}</div><div class="card"><h2>PARTE 5: TÉCNICA DE FEYNMAN (APRENDA RÁPIDO)</h2>{html_feynman}</div>
<div class="card"><h2>PARTE 6: GLOSSÁRIO REMISSIVO</h2>{html_glossario}</div><div class="card"><h2>PARTE 7: QUESTÕES DE CONCURSO</h2>{html_questoes}</div>
<div class="card"><h2>PARTE 8: MAPA MENTAL</h2>{html_mapa}</div></div></body></html>
"""
                with open(nome_do_arquivo, "w", encoding="utf-8") as f:
                    f.write(conteudo_html)
                print(f"\n✅ Arquivo '{nome_do_arquivo}' foi gerado com sucesso!")
                print("-> Iniciando o download do arquivo. Por favor, verifique seu navegador.")
                files.download(nome_do_arquivo)
                print("\n\n--- FIM DO PROCESSO ---")
            except Exception as e:
                print(f"❌ Ocorreu um erro geral durante a execução das análises: {e}")
            finally:
                button.disabled = False

    button.on_click(iniciar_analise)

    print("--- PRONTO PARA COMEÇAR ---")
    print("Cole o texto que deseja analisar na caixa abaixo e clique no botão 'Analisar Texto'.")
    display(textarea, button, output_area)

else:
    print("\nO robô foi interrompido porque as chaves de API não foram configuradas. Por favor, siga as instruções na mensagem de erro acima.")


--- PARTE 1: CONFIGURANDO O AMBIENTE ---
✅ Chave da API do Google AI (Gemini) configurada.
✅ Chave da API de Busca (Google Custom Search) configurada.
✅ ID do Mecanismo de Busca configurado.
Mounted at /content/drive
✅ Google Drive conectado.
--- Configuração do ambiente concluída! ---

--- PARTE 2: LENDO OS ARQUIVOS DO SEU DRIVE ---

🟡 Nenhum arquivo de fonte foi listado. A análise usará apenas o conhecimento geral da IA.
--- Leitura dos arquivos concluída! ---

--- PARTE 3: PREPARANDO TODAS AS FERRAMENTAS DO ROBÔ ---

✅ Ferramenta de IA ('models/gemini-pro-latest') configurada!
✅ Todas as ferramentas do robô estão prontas!

--- PRONTO PARA COMEÇAR ---
Cole o texto que deseja analisar na caixa abaixo e clique no botão 'Analisar Texto'.


Textarea(value='', description='Texto:', layout=Layout(height='250px', width='99%'), placeholder='Cole aqui o …

Button(button_style='success', description='Analisar Texto', icon='cogs', style=ButtonStyle(), tooltip='Clique…

Output()